What kind of analysis can I actually do? 
  * a basic sum of week over week



#### MVP: 
 * a streamlit dashboard and a tableau dashboard
 * filter by activities
 * **total time spent on activites by week**

#### ideas for this data: 
 * convert the map data to elevation data
 * something with the suffer score
   * suffer per second? suffer per minute? 

#### to do
 * convert start_date column to pandas timestamp 

In [1]:
import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

In [5]:
def get_strava_data(csv_file):
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.read_csv(csv_file)

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')     
               
        # if there are no new values (i.e. activities) found in r_df, just break.  otherwise keep going
        if len([v for v in list(r_df['id']) if v not in list(activities['id'])]) == 0:
            activities.drop_duplicates(subset=['id'], inplace=True)
            activities.sort_values(by='id', ascending=False, inplace=True)
            activities.to_csv(csv_file, index=False)
            return activities

        if (not r):   # if no results then exit loop
            break
            
        activities = pd.concat([r_df, activities])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)
    activities.sort_values(by='id', ascending=False, inplace=True)
    activities.to_csv(csv_file, index=False)
    return activities

In [2]:
#get_strava_data('strava_activities.csv')

In [8]:
cols_to_drop = ['Unnamed: 0.1', 'athlete', 'resource_state', 'start_date', 'timezone', 
'utc_offset', 'trainer', 'commute', 'type', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
'average_watts', 'kilojoules', 'device_watts', 'photo_count', 'average_temp', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
'has_heartrate', 'location_city', 'location_state', 'location_country', 'private', 'has_kudoed', 'Unnamed: 0']

strava_activities_raw = pd.read_csv('strava_activities.csv')

strava_activities_clean = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

# meters to miles
strava_activities_clean['distance'] = strava_activities_clean.distance * .000621

# might be a fun metric to plot over time. perhaps my suffering per minute has gone down as ive gotten fitter
strava_activities_clean['SPM'] = strava_activities_clean.suffer_score / (strava_activities_clean.moving_time / 60)

rename = {'start_date_local': 'start_time'}
strava_activities_clean.rename(rename, axis=1, inplace=True)

# convert start_time to pandas Timestamp object. then we can access seconds, minutes, hours, etc. and can add the moving_time
strava_activities_clean['start_time'] = pd.to_datetime(strava_activities_clean['start_time'])

strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time'] = pd.Timestamp(year=2022, month=6, day=12, hour=2, minute=8, second=36, tz='UTC')

strava_activities_clean.set_index('start_time', inplace=True)

In [124]:
#strava_activities_clean.index[17] = strava_activities_clean.index[17].replace(day=12)

In [4]:
# my 6/12 run is showing up as 6/13

st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(15)
#st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('M', closed='left').distance.sum().tail(15)

#st.index = st.index.strftime('%B %Y')
st.index = st.index.strftime('Week of %A, %b %d')

In [5]:
#st.index = st.index.strftime('%B %Y')
st.tail(5)

start_time
Week of Monday, Jun 20    21.896833
Week of Monday, Jun 27    42.420821
Week of Monday, Jul 04    26.436405
Week of Monday, Jul 11    41.725922
Week of Monday, Jul 18     6.080211
Name: distance, dtype: float64

In [9]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)
strava_activities_clean.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'sport_type', 'id', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'map', 'average_speed',
       'max_speed', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'pr_count', 'suffer_score', 'average_cadence', 'SPM'],
      dtype='object')

In [16]:
strava_activities_raw.start_latlng[109]

'[45.52732522599399, -122.60262675583363]'

In [106]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [107]:
fig.show()

In [76]:
type(strava_activities_clean['start_time'][2])

str

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [67]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')